## Dreem 2 Sleep Classification challenge 2020
**Student: Felipe Cybis Pereira**

This notebook produces multitaper spectrograms for each epoch and saves them as a new dataset to be used.

By commiting this notebook, the output will be saved in Kaggle and will be possible to use as input data in a different Kaggle notebook.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import h5py # Read and write HDF5 files from Python

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# filenames
data_path = "/kaggle/input/dreem-2-sleep-classification-challenge-2020/"
file_xtrain = data_path + "X_train.h5/X_train.h5"
file_xtest = data_path + "X_test.h5/X_test.h5"
file_ytrain = data_path + "y_train.csv"

! pip install lspopt

/kaggle/input/dreem-2-sleep-classification-challenge-2020/sample_submission.csv
/kaggle/input/dreem-2-sleep-classification-challenge-2020/y_train.csv
/kaggle/input/dreem-2-sleep-classification-challenge-2020/X_train.h5/X_train.h5
/kaggle/input/dreem-2-sleep-classification-challenge-2020/X_test.h5/X_test.h5
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
from lspopt import spectrogram_lspopt # function needed for multitaper spectrogram

def normalize_data(eeg_array):
    """Normalizes signal between -1 and 1 by clipping data between -400 and 400 micro Volts"""

    normalized_array = np.clip(eeg_array, -400, 400)
    normalized_array = normalized_array / 400

    return normalized_array

def compute_spectrogram(eeg_data,
                        fs = 50.,
                        win_sec = 2.,
                        fmin = 0.5,
                        fmax = 20.,
                        sec_overlap = 1.5,
                        c_parameter=20.0):
        """
        Compute spectrogram from EEG 1D-array
            :param eeg_data (numpy 1D array): 1D numpy array of raw eeg data
            :param fs (float): sampling frequency (default = 50 Hz)
            :param win_sec (float): time of the taper windows (default = 2 sec)
            :param fmin (float): lower bound of the frequencies to be returned in the spectrogram (default = 0.5 Hz)
            :param fmax (float): higher bound of the frequencies to be returned in the spectrogram (default = 20.0 Hz)
            :param sec_overlap (float): overlapping window in seconds (default = 1.5 sec)
            :param c_parameter (float): C parameter as defined in doi:10.1155/2011/980805
            
            :return Sxx (numpy 2D array): Multitaper spectrogram
            :return t (numpy 1D array): corresponding time array of the spectrogram
            :return f (numpy 1D array): corresonding frequency array of the spectrogram
        """
        
        # Calculate multi-taper spectrogram
        nperseg = int(win_sec * fs)
        noverlap = sec_overlap * fs
        assert eeg_data.size > 2 * nperseg, 'Data length must be at least 2 * win_sec * fs (' +str(nperseg)+ '). It is ' + str(eeg_data.size)
        f, t, Sxx = spectrogram_lspopt(eeg_data, fs, c_parameter, nperseg=nperseg, noverlap=noverlap)

        Sxx = 10 * np.log10(Sxx)  # Convert uV^2 / Hz --> dB / Hz
        Sxx[Sxx == np.inf] = 0
        Sxx[Sxx == -np.inf] = 0

        # Select only relevant frequencies (up to 30 Hz)
        good_freqs = np.logical_and(f >= fmin, f <= fmax)
        Sxx = Sxx[good_freqs, :]
        f = f[good_freqs]

        return Sxx, t, f
    
def get_data(file_path, derivation):
    """Get .h5 data from path"""
    
    with h5py.File(file_path, "r") as fi:
        data = fi[derivation][()]
        
    return data
    
def get_spectrograms(file_path, channel_list=['eeg_1', 'eeg_2', 'eeg_3', 'eeg_4', 'eeg_5', 'eeg_6', 'eeg_7']):
    """Uses compute_spectrogram function to create new dataset of spectrograms
        :param file_path (path): file path to get eeg raw data
        :param channel_list (list): list of derivations to get from .h5 eeg file
        
        :return dict_with_spectrograms (dict): python dictionaire with spectrograms from `channel_list` as well as `index` and `index_window`
    """
    
    index = get_data(file_path, 'index')
    index_window = get_data(file_path, 'index_window')
    
    # Init python dictionaire with keys for spectrograms
    keys = ['Sxx_1', 'Sxx_2', 'Sxx_3', 'Sxx_4', 'Sxx_5', 'Sxx_6', 'Sxx_7']
    dict_with_spectrograms = dict.fromkeys(keys)
    dict_with_spectrograms['index'] = index
    dict_with_spectrograms['index_window'] = index_window
        
    for channel, key in zip(channel_list, keys):
        x_data = get_data(file_path, channel) # eeg raw data
        subjects = np.unique(index) # subjects in training set
        
        list_of_Sxx = []
        for subject in subjects:
            print('Getting spectrogram from ' + channel + '. Subject ' + str(subject) + '...', end='\r')
            subject_data = x_data[index==subject]
            
            ## Params to compute spectrogram
            fs = 50.
            win_sec = 2.
            fmin = 0.5
            fmax = 20.
            sec_overlap = 1.5
            step_size = win_sec - sec_overlap
            ## these parameters were chosen to result in spectrograms of 30 seconds of size (40x60)
            
            padding_size = int(win_sec / 2 / step_size)
            Sxx_epoch_size = int(30 // step_size) # think about [0., 0.5, ..., 29.5]
            
            assert (win_sec / 2) % step_size == 0, 'For clean data shape, win_sec/2 should be div by step_size'
            Sxx, _, _ = compute_spectrogram(subject_data.flatten(), fs, win_sec, fmin, fmax, sec_overlap)
            
            ## padding before and after spectrogram so that every epoch can have same dimensions!
            padding_before = np.zeros((Sxx.shape[0], padding_size))
            padding_after = np.zeros((Sxx.shape[0], padding_size-1))
            
            new_Sxx = np.append(padding_before, np.append(Sxx, padding_after, axis=1), axis=1)
            assert new_Sxx.shape[-1] % Sxx_epoch_size == 0, 'Spectrogram has not the right shape, Sxx.shape=' + str(new_Sxx.shape)
            
            ## Sxx has shape (frequency_bins, time_bins) with time_bins going from first to last subject's epoch
            new_Sxx_reshape = new_Sxx.reshape(new_Sxx.shape[0], subject_data.shape[0], -1)
            ## now Sxx has shape (frequency_bins, num_epochs, time_bins) with time_bins for each epoch
            new_Sxx_transpose = new_Sxx_reshape.transpose((1,0,2))
            ## now Sxx has shape (num_epochs, frequency, time_bins)
            list_of_Sxx += [new_Sxx_transpose]
            
        dict_with_spectrograms[key] = np.concatenate(list_of_Sxx, axis=0)
        # now if the eeg data has 24688 epochs, each epoch with 1500 data points
        # the Sxx data will have 24688 epochs, each epoch with (frequency_bins,time_bins) dimensions
        # here Sxx will have dimensions (24688, 40, 60)
    
    return dict_with_spectrograms
            
def save_dict_as_h5py(dictionaire, filename):
    """Save python dictionaire as .h5 file"""
    working_dir = os.getcwd()
    
    hf = h5py.File(os.path.join(working_dir, filename), 'w')
    
    for key, value in dictionaire.items():
        hf.create_dataset(key, data=value)
    
    hf.close()


In [3]:
# Computing spectrograms of the training data
dict_with_spectrograms = get_spectrograms(file_xtrain)
save_dict_as_h5py(dict_with_spectrograms, 'Sxx_x_train.h5')

In [4]:
# Computing spectrograms of the test data
dict_with_spectrograms = get_spectrograms(file_xtest)
save_dict_as_h5py(dict_with_spectrograms, 'Sxx_x_test.h5')

In [5]:
# saving y data with the spectrograms as well, just to have all together 
y_data = pd.read_csv(file_ytrain) 
y_data.to_csv('y_train.csv', index=False)